# LSTM, GRU을 이용한 텍스트 생성
- 데이터 다운로드 : 캐글 New York Times Comments
- ArticlesApril2018.csv

In [1]:
import numpy as np
import tensorflow as tf 
seed = 2021
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical   


In [3]:
import pandas as pd
df = pd.read_csv('data/ArticlesApril2018.csv', encoding='latin1')
df.head()

,articleID,articleWordCount,byline,documentType,headline,keywords,multimedia,newDesk,printPage,pubDate,sectionName,snippet,source,typeOfMaterial,webURL
0,5adf6684068401528a2aa69b,781,By JOHN BRANCH,article,Former N.F.L. Cheerleadersâ Settlement Offer...,"['Workplace Hazards and Violations', 'Football...",68,Sports,0,2018-04-24 17:16:49,Pro Football,"âI understand that they could meet with us, ...",The New York Times,News,https://www.nytimes.com/2018/04/24/sports/foot...
1,5adf653f068401528a2aa697,656,By LISA FRIEDMAN,article,E.P.A. to Unveil a New Rule. Its Effect: Less ...,"['Environmental Protection Agency', 'Pruitt, S...",68,Climate,0,2018-04-24 17:11:21,Unknown,The agency plans to publish a new regulation T...,The New York Times,News,https://www.nytimes.com/2018/04/24/climate/epa...
2,5adf4626068401528a2aa628,2427,By PETE WELLS,article,"The New Noma, Explained","['Restaurants', 'Noma (Copenhagen, Restaurant)...",66,Dining,0,2018-04-24 14:58:44,Unknown,Whatâs it like to eat at the second incarnat...,The New York Times,News,https://www.nytimes.com/2018/04/24/dining/noma...
3,5adf40d2068401528a2aa619,626,By JULIE HIRSCHFELD DAVIS and PETER BAKER,article,Unknown,"['Macron, Emmanuel (1977- )', 'Trump, Donald J...",68,Washington,0,2018-04-24 14:35:57,Europe,President Trump welcomed President Emmanuel Ma...,The New York Times,News,https://www.nytimes.com/2018/04/24/world/europ...
4,5adf3d64068401528a2aa60f,815,By IAN AUSTEN and DAN BILEFSKY,article,Unknown,"['Toronto, Ontario, Attack (April, 2018)', 'Mu...",68,Foreign,0,2018-04-24 14:21:21,Canada,"Alek Minassian, 25, a resident of Torontoâs ...",The New York Times,News,https://www.nytimes.com/2018/04/24/world/canad...


## 데이터 전처리

In [4]:
df.columns

Index(['articleID', 'articleWordCount', 'byline', 'documentType', 'headline',
       'keywords', 'multimedia', 'newDesk', 'printPage', 'pubDate',
       'sectionName', 'snippet', 'source', 'typeOfMaterial', 'webURL'],
      dtype='object')

In [5]:
# headline 열 데이터만 사용, Null 검사
df.headline.isnull().sum()

0

In [6]:
headlines = [text for text in df.headline.values]
len(headlines)

1324

In [7]:
headlines[:5]  #데이터 확인

['Former N.F.L. Cheerleadersâ\x80\x99 Settlement Offer: $1 and a Meeting With Goodell',
 'E.P.A. to Unveil a New Rule. Its Effect: Less Science in Policymaking.',
 'The New Noma, Explained',
 'Unknown',
 'Unknown']

In [8]:
# 노이즈 데이터('unknown')제거
headlines = [text for text in headlines if text != 'Unknown']
len(headlines)

1214

In [9]:
# 구둣점 제거, 소문자로 변환
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
def preprocessing(s):
    s = s.encode('utf8').decode('ascii','ignore')
    return ''.join(c for c in s if c not in punctuation).lower()

In [11]:
headlines = [preprocessing(s) for s in headlines]
headlines[:3]

['former nfl cheerleaders settlement offer 1 and a meeting with goodell',
 'epa to unveil a new rule its effect less science in policymaking',
 'the new noma explained']

In [12]:
# 단어 집합을 만들고, 그 크기를 확인
t = Tokenizer()
t.fit_on_texts(headlines)
vocab_size = len(t.word_index) +1
vocab_size 

3494

In [13]:
# 학습에 사용될 샘플 시퀀스
sequences = []
for line in headlines:
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence= encoded[: i+1]
        sequences.append(sequence)
sequences[:11]

[[99, 269],
 [99, 269, 371],
 [99, 269, 371, 1115],
 [99, 269, 371, 1115, 582],
 [99, 269, 371, 1115, 582, 52],
 [99, 269, 371, 1115, 582, 52, 7],
 [99, 269, 371, 1115, 582, 52, 7, 2],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10],
 [99, 269, 371, 1115, 582, 52, 7, 2, 372, 10, 1116],
 [100, 3]]

In [14]:
# 최대 길이 찾기 # 패딩씌우려고
max_len = max(len(s) for s in sequences ) 
max_len

24

In [15]:
sequences = pad_sequences(sequences, maxlen= max_len, padding = 'pre') #패딩 앞에 씌움
sequences[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          99,  269],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   99,
         269,  371],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   99,  269,
         371, 1115]], dtype=int32)

In [16]:
X = sequences[:,:-1]
y = sequences[:,-1]
Y = to_categorical(y)  #to_categorical(y, vocab_size)
X.shape, Y.shape

((7803, 23), (7803, 3494))

In [17]:
'former nfl cheerleaders settlement offer 1 and a meting with goodell'

'former nfl cheerleaders settlement offer 1 and a meting with goodell'

## 모델 정의 / 설정 / 학습
- Embedding
- LSTM
- GRU

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense
from my_util import sentence_generation

In [19]:
def execute_model_LSTM(n_embed): 
    model = Sequential([
    Embedding(vocab_size,n_embed, input_length= max_len -1), # 4: 4차원
    LSTM(128),
    Dense(vocab_size, activation='softmax')
    ])
    model.compile(
    optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy']
    )
    history = model.fit(X, Y, epochs =200, verbose =0 )
    print(f"정확도: {history.history['accuracy'][-1] }")
    print(sentence_generation(model, t, max_len, 'i',10))
    print(sentence_generation(model,t, max_len,'epa',10))
    print(sentence_generation(model, t, max_len, 'former',10))

In [20]:
def execute_model_GRU(n_embed): 
    model = Sequential([
    Embedding(vocab_size,n_embed, input_length= max_len -1), # 4: 4차원
    GRU(128),
    Dense(vocab_size, activation='softmax')
    ])
    model.compile(
    optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy']
    )
    history = model.fit(X, Y, epochs =200, verbose =0 )
    print(f"정확도: {history.history['accuracy'][-1] }")
    print(sentence_generation(model, t, max_len, 'i',10))
    print(sentence_generation(model,t, max_len,'epa',10))
    print(sentence_generation(model, t, max_len, 'former',10))

In [21]:
for n_embed in [8,10,12]:
     print('==============LSTM============')
     print(f' LSTM_128 :  n_embed={n_embed}')
     execute_model_LSTM(n_embed)
     print('==============GRU===============')
     print(f' GRU_128 :  n_embed={n_embed}')
     execute_model_GRU(n_embed)
     print()

==============LSTM============
 LSTM_128 :  n_embed=8
정확도: 0.9175958037376404
i want to be rich and im not sorry work about
epa sheriff indulged pruitt as security spending mounted work law plans
former nfl cheerleaders settlement offer 1 and a meeting with goodell
==============GRU===============
 GRU_128 :  n_embed=8
정확도: 0.9125977158546448
i want to be rich and im not sorry control on
epa sheriff indulged pruitt as security spending mounted left i can
former judge to review files seized in searches our crash usually

==============LSTM============
 LSTM_128 :  n_embed=10
정확도: 0.9174676537513733
i want to be rich and im not sorry duties on
epa sheriff indulged pruitt as security spending mounted feeling now as
former nfl cheerleaders settlement offer 1 and a meeting with goodell
==============GRU===============
 GRU_128 :  n_embed=10
정확도: 0.9163142442703247
i want to be rich and im not sorry king home
epa sheriff indulged pruitt as security spending mounted left home didnt
former nfl